## Basic Ngram Model

The code in this notebook is inspired by the code from this source: https://towardsdatascience.com/text-generation-using-n-gram-model-8d12d9802aa0. I did change quite a few things from the source code, which I commented on throughout my code. Right now, I implemented the model as a trigram model, but you can implement it as any ngram model.

In [11]:
import numpy as np
import pandas as pd
import nltk
import string
import random
import torch
import torch.nn as nn
import random
import time
from typing import List

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

if torch.backends.cudnn.enabled:
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed_all(SEED)

In [12]:
df_train = pd.read_csv("../input/lyrics/lyrics.csv") #in kaggle
#df_train = pd.read_csv("data/lyrics.csv") 

In [13]:
i=0
pop_lyrics = list()
while i < len(df_train.index):
    if df_train['genre'][i] == 'Pop' and type(df_train['lyrics'][i]) == str:
        pop_lyrics.append(df_train['lyrics'][i])
    i += 1

In [14]:
stop = set(nltk.corpus.stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = nltk.stem.wordnet.WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.split() if i not in stop])
    punc_free = "".join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    out = normalized.lower().split()
    return out

Note that in the code below I removed the tokenize function from the original code, because I already lemmatized the sentence in the same way as we did for the neural network. I used "start" and "stop" to generated the n-grams, so a bit different from the neural network. Furthermore, I changed a few things in the word generation so it would start generating from two given words instead of from randomly initialized words. Finally, I did not split the sentences at periods, since this doesn't really make sense for songs. Instead, I split the input at every new song. 

In [15]:
def get_ngrams(n: int, tokens: list) -> list:
    """
    :param n: n-gram size
    :param tokens: tokenized sentence
    :return: list of ngrams
    ngrams of tuple form: ((previous wordS!), target word)
    """
    tokens = (n-1)*['<START>']+tokens+(n-1)*['<END>']
    l = [(tuple([tokens[i-p-1] for p in reversed(range(n-1))]), tokens[i]) for i in range(n-1, len(tokens))]
    return l


class NgramModel(object):

    def __init__(self, n):
        self.n = n
        # dictionary that keeps list of candidate words given context
        self.context = {}
        # keeps track of how many times ngram has appeared in the text before
        self.ngram_counter = {}

    def update(self, sentence: str) -> None:
        """
        Updates Language Model
        :param sentence: input text
        """
        n = self.n
        ngrams = get_ngrams(n, clean(sentence))
        for ngram in ngrams:
            if ngram in self.ngram_counter:
                self.ngram_counter[ngram] += 1.0
            else:
                self.ngram_counter[ngram] = 1.0

            prev_words, target_word = ngram
            if prev_words in self.context:
                self.context[prev_words].append(target_word)
            else:
                self.context[prev_words] = [target_word]

    def prob(self, context, token):
        """
        Calculates probability of a candidate token to be generated given a context
        :return: conditional probability
        """
        try:
            count_of_token = self.ngram_counter[(context, token)]
            count_of_context = float(len(self.context[context]))
            result = count_of_token / count_of_context

        except KeyError:
            result = 0.0
        return result

    def random_token(self, context):
        """
        Given a context we "semi-randomly" select the next word to append in a sequence
        :param context:
        :return:
        """
        r = random.random()
        map_to_probs = {}
        token_of_interest = self.context[context]
        for token in token_of_interest:
            map_to_probs[token] = self.prob(context, token)

        summ = 0
        for token in sorted(map_to_probs):
            summ += map_to_probs[token]
            if summ > r:
                return token

    def generate_text(self, context_queue, token_count: int):
        """
        :param token_count: number of words to be produced
        :return: generated text
        """
        n = self.n
        result = []
        for _ in range(token_count):
            obj = self.random_token(tuple(context_queue))
            result.append(obj)
            context_queue.pop(0)
            context_queue.append(obj)
        return ' '.join(result)


def create_ngram_model(n, intext):
    m = NgramModel(n)
    for sentence in intext:
        m.update(sentence)
    return m


if __name__ == "__main__":
    start = time.time()
    m = create_ngram_model(3, pop_lyrics) 
    print (f'Language Model creating time: {time.time() - start}')
    start = time.time()
    random.seed(7)
    print(f'{"="*50}\nGenerated text:')
    print(m.generate_text(["i", "am"], 30)) #change this if you want different input words/a different length
    print(f'{"="*50}')

Language Model creating time: 61.217856645584106
Generated text:
i crazy might go go go away if asked me do do do oh bisasha bisasha hey he said how walk away walk away but everybody know truth hard lie
